In [1]:
import os
import subprocess
from pathlib import Path

platform = os.getenv('WORDSLAB_PLATFORM', 'UnknownLinux')
platform

'WindowsSubsystemForLinux'

In [3]:
# Get Ubuntu version
try:
    # Try reading from /etc/os-release first
    with open('/etc/os-release', 'r') as f:
        for line in f:
            if line.startswith('PRETTY_NAME='):
                os_version = line.split('=')[1].strip().strip('"')
                break
except:
    try:
        # Fallback to reading from /etc/issue
        with open('/etc/issue', 'r') as f:
            os_version = f.read().strip().split('\\')[0].strip()
    except:
        os_version = "Unknown"

os_version

'Ubuntu 24.04.1 LTS'

In [4]:
wordslab_version = os.getenv('WORDSLAB_VERSION', 'Unknown')
wordslab_version

'2025-01'

In [5]:
from importlib.metadata import version
import torch
import pynvml

apps_versions = { 'pytorch': version('torch') }

cpu_only_file = Path(os.getenv('WORDSLAB_NOTEBOOKS_ENV', '')) / '.cpu-only'
if not cpu_only_file.exists():
    try:
        if torch.cuda.is_available():
            apps_versions['python cuda'] = torch.version.cuda
    except:
        pass
    try:
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        driver_cuda_version = pynvml.nvmlSystemGetCudaDriverVersion_v2()
        apps_versions['gpu cuda'] = f"{driver_cuda_version//1000}.{(driver_cuda_version%1000)//10}"
        pynvml.nvmlShutdown()
    except:
        pass

apps_versions

{'pytorch': '2.5.1', 'python cuda': '12.4', 'gpu cuda': '12.7'}

### Get URL - Windows Subsystem for Linux

```
User req as a route parameter
=> req.url

Append port : 8880 - 8888
```

### Get URL - Jarvislabs

```
MACHINE_NAME=test-install2 (UI)
MACHINE_ID=261560 (API)

HOSTNAME=98708c9cbe44 (?)

reserved ports: 6006,7007

Native JUPYTERLAB
internal port: 8889
https://98708c261555.notebooks.jarvislabs.net/lab

APIs
https://98708c2615551.notebooks.jarvislabs.net/ => 6006
https://98708c2615552.notebooks.jarvislabs.net/ => Jupyterlab:8880
https://98708c2615553.notebooks.jarvislabs.net/ => VsCode:8881
https://98708c2615553.notebooks.jarvislabs.net/ => Ollama:8882
...
https://98708c26155510.notebooks.jarvislabs.net/ => Dashboard:8888
```

### Get URL - Runpod

```
RUNPOD_POD_ID=ebkhet9vq5flt7 (UI + used for URLs)

RUNPOD_PUBLIC_IP=69.30.85.49
RUNPOD_TCP_PORT_22=22047

RUNPOD_DC_ID=CA-MTL-1
RUNPOD_POD_HOSTNAME=ebkhet9vq5flt7-644112e4

HOSTNAME=5936ba7c007d (?)

Native JUPYTERLAB
internal port: 8888
https://ebkhet9vq5flt7-8888.proxy.runpod.net/lab

Http Services
https://ebkhet9vq5flt7-8880.proxy.runpod.net/
...
https://ebkhet9vq5flt7-8888.proxy.runpod.net/
```

### Get URL - Vast.ai

```
CONTAINER_ID=16855978 (UI)

HOSTNAME=9e254447c27d

PUBLIC_IPADDR=142.115.158.140
VAST_TCP_PORT_22=42790
VAST_TCP_PORT_8080=42702

Native JUPYTERLAB
internal port: 8080
https://142.115.158.140:42702/tree

Open ports
VAST_TCP_PORT_8880=42777
VAST_TCP_PORT_8881=42791
VAST_TCP_PORT_8882=42623
VAST_TCP_PORT_8883=42779
VAST_TCP_PORT_8884=42668
VAST_TCP_PORT_8885=42663
VAST_TCP_PORT_8886=42729
VAST_TCP_PORT_8887=42736
```

In [42]:
def get_root_urls(platform, req):

    if platform=="WindowsSubsystemForLinux" or platform=="UnknownLinux":
        
        dashboard_root_url = f"{req.url.scheme}://{req.url.host}"
        return [f"{dashboard_root_url}:{port}" for port in range(8880,8889)]
    
    elif platform=="Jarvislabs.ai":

        url_prefix, url_suffix = req.url.host.split('.', 1)
        return [f"{req.url.scheme}://{url_prefix}{i}.{url_suffix}" for i in range(2,11)]

    elif platform=="Runpod.io": 

        pod_id = os.getenv('RUNPOD_POD_ID')
        _, domain = req.url.host.split('.', 1)    
        return [f"{req.url.scheme}://{pod_id}-{port}.{domain}" for port in range(8880,8889)]

    elif platform=="Vast.ai": 

        public_ip = os.getenv('PUBLIC_IPADDR')
        external_ports = [os.getenv(f'VAST_TCP_PORT_{port}') for port in range(8880,8889)]
        return [f"{req.url.scheme}://{public_ip}:{ext_port}" for ext_port in external_ports]

In [47]:
from types import SimpleNamespace

reqWSL = lambda: None
reqWSL.url = lambda: None
reqWSL.url.scheme = "http"
reqWSL.url.host = "127.0.0.1"

reqJarvis = lambda: None
reqJarvis.url = lambda: None
reqJarvis.url.scheme = "https"
reqJarvis.url.host = "98708c261555.notebooks.jarvislabs.net"

reqRunpod = lambda: None
reqRunpod.url = lambda: None
reqRunpod.url.scheme = "https"
reqRunpod.url.host = "ebkhet9vq5flt7-8888.proxy.runpod.net"

os.environ["RUNPOD_POD_ID"] = "ebkhet9vq5flt7"

reqVast = lambda: None
reqVast.url = lambda: None
reqVast.url.scheme = "https"
reqVast.url.host = "142.115.158.140"

os.environ["PUBLIC_IPADDR"] = "142.115.158.140"
os.environ["VAST_TCP_PORT_8880"] = "42777"
os.environ["VAST_TCP_PORT_8881"] = "42791"
os.environ["VAST_TCP_PORT_8882"] = "42623"
os.environ["VAST_TCP_PORT_8883"] = "42779"
os.environ["VAST_TCP_PORT_8884"] = "42668"
os.environ["VAST_TCP_PORT_8885"] = "42663"
os.environ["VAST_TCP_PORT_8886"] = "42729"
os.environ["VAST_TCP_PORT_8887"] = "42736"

In [27]:
get_root_urls("WindowsSubsystemForLinux", reqWSL)

['http://127.0.0.1:8880',
 'http://127.0.0.1:8881',
 'http://127.0.0.1:8882',
 'http://127.0.0.1:8883',
 'http://127.0.0.1:8884',
 'http://127.0.0.1:8885',
 'http://127.0.0.1:8886',
 'http://127.0.0.1:8887',
 'http://127.0.0.1:8888']

In [31]:
get_root_urls("Jarvislabs.ai", reqJarvis)

['https://98708c2615552.notebooks.jarvislabs.net',
 'https://98708c2615553.notebooks.jarvislabs.net',
 'https://98708c2615554.notebooks.jarvislabs.net',
 'https://98708c2615555.notebooks.jarvislabs.net',
 'https://98708c2615556.notebooks.jarvislabs.net',
 'https://98708c2615557.notebooks.jarvislabs.net',
 'https://98708c2615558.notebooks.jarvislabs.net',
 'https://98708c2615559.notebooks.jarvislabs.net',
 'https://98708c26155510.notebooks.jarvislabs.net']

In [40]:
get_root_urls("Runpod.io", reqRunpod)

['https://ebkhet9vq5flt7-8880.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8881.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8882.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8883.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8884.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8885.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8886.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8887.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8888.proxy.runpod.net']

In [48]:
get_root_urls("Vast.ai", reqVast)

['https://142.115.158.140:42777',
 'https://142.115.158.140:42791',
 'https://142.115.158.140:42623',
 'https://142.115.158.140:42779',
 'https://142.115.158.140:42668',
 'https://142.115.158.140:42663',
 'https://142.115.158.140:42729',
 'https://142.115.158.140:42736',
 'https://142.115.158.140:None']